In [1]:
import os
import cv2
import uuid
import time
import fitz
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)

In [2]:
file_path = "./data/insurance_biz.pdf"
doc = fitz.open(file_path)

In [3]:
folder_path = "./data/page_imgs"
os.makedirs(folder_path, exist_ok=True)

for idx, page in enumerate(doc):
    img = page.get_pixmap()
    img.save(f"{folder_path}/{idx:04d}.jpg")

In [4]:
img_list = [img for img in os.listdir('./data/page_imgs') if img.endswith('.jpg')]
img_list = sorted(img_list)

print(img_list)

['0000.jpg', '0001.jpg', '0002.jpg', '0003.jpg', '0004.jpg', '0005.jpg', '0006.jpg', '0007.jpg', '0008.jpg', '0009.jpg', '0010.jpg', '0011.jpg', '0012.jpg', '0013.jpg', '0014.jpg', '0015.jpg', '0016.jpg', '0017.jpg', '0018.jpg', '0019.jpg', '0020.jpg', '0021.jpg', '0022.jpg', '0023.jpg', '0024.jpg', '0025.jpg', '0026.jpg', '0027.jpg', '0028.jpg', '0029.jpg', '0030.jpg', '0031.jpg', '0032.jpg', '0033.jpg', '0034.jpg', '0035.jpg', '0036.jpg', '0037.jpg', '0038.jpg', '0039.jpg', '0040.jpg', '0041.jpg', '0042.jpg', '0043.jpg']


In [5]:
secret_key = "YWVQcGxYdVBIZkZlYWdkb3BLaWFkS1BDZUFHUWpUZmw="
api_url = "https://2n0z4f9xv9.apigw.ntruss.com/custom/v1/36587/03dfe0844a4f0f91709228b1daee20069952854d05ce94c1aabb7089b789e481/general"

os.makedirs("./data/json", exist_ok=True)
string_result = ""
for image_file in img_list:
    request_json = {
        'images' : [{'format' : 'jpg', 'name' : 'demo'}],
        'requestId' : str(uuid.uuid4()),
        'version' : 'V2',
        'timestamp' : int(round(time.time() * 1000))
    }
    payload = {'message' : json.dumps(request_json).encode("UTF-8")}
    files = [('file', open(f"./data/page_imgs/{image_file}", 'rb'))]
    headers = {'X-OCR-SECRET' : secret_key}
    response = requests.request("POST", api_url, headers=headers, data = payload, files = files)

    json_data = response.json()
    for i in json_data['images'][0]['fields']:
        if i['lineBreak'] == True:
            linebreak = ' '
        else:
            linebreak = ' '
        string_result = string_result + i['inferText'] + linebreak

    print(image_file)
    json_file_path = f"./data/json/{image_file.replace('.jpg', '')}.json"
    with open(json_file_path, 'w', encoding='utf-8') as file:
        json.dump(json_data, file, ensure_ascii=False, indent=4)

0000.jpg
0001.jpg
0002.jpg
0003.jpg
0004.jpg
0005.jpg
0006.jpg
0007.jpg
0008.jpg
0009.jpg
0010.jpg
0011.jpg
0012.jpg
0013.jpg
0014.jpg
0015.jpg
0016.jpg
0017.jpg
0018.jpg
0019.jpg
0020.jpg
0021.jpg
0022.jpg
0023.jpg
0024.jpg
0025.jpg
0026.jpg
0027.jpg
0028.jpg
0029.jpg
0030.jpg
0031.jpg
0032.jpg
0033.jpg
0034.jpg
0035.jpg
0036.jpg
0037.jpg
0038.jpg
0039.jpg
0040.jpg
0041.jpg
0042.jpg
0043.jpg


In [6]:
df = pd.DataFrame({'text' : [string_result,]})

In [7]:
df.to_excel('./data/corpus.xlsx', index=False)

In [8]:
df

,text
0,보험약관 카드회원단체상해보험 목차 보통약관 특별약관 해외여행중 배상책임 특별약관 국...


In [9]:
string_result

'보험약관 카드회원단체상해보험 목차 보통약관 특별약관 해외여행중 배상책임 특별약관 국내여행중 공공교통승용구탑승중 배상책임 특별약관 상해사망후유장해 부보장 특별약관 지정대리청구서비스 특별약관 카드회원 보험료정산 추가약관 해외발생 실손의료비(카드회단체) 특별약관 【보통약관】 제1관 목적 및 용어의 정의 제1조(목적) 이 보험계약(이하 \'계약\' 이라 합니다)은 보험계약자(이하 \'계약자\' 라 합니다)와 보험회사(이하 \'회사\' 라 합니다) 사이에 피보험자의 상해에 대한 위험을 보장하기 위하여 체결됩니다. 제2조(용어의 정의) 이 계약에서 사용되는 용어의 정의는, 이 계약의 다른 조항에서 달리 정의되지 않는 한 다음과 같습니다. 1. 계약관계 관련 용어 가. 계약자: 회사와 계약을 체결하고 보험료를 납입할 의무를 지는 사람을 말합니다. 나. 보험수익자: 보험금 지급사유가 발생하는 때에 회사에 보험금을 청구하여 받을 수 있는 사람을 말합니다. 다. 보험증권: 계약의 성립과 그 내용을 증명하기 위하여 회사가 계약자에게 드리는 증서를 말합니다. 라. 진단계약: 계약을 체결하기 위하여 피보험자가 건강진단을 받아야 하는 계약을 말합니다. 마. 피보험자: 보험사고의 대상이 되는 사람을 말합니다. 2. 지급사유 관련 용어 가. 상해: 보험기간 중에 발생한 급격하고도 우연한 외래의 사고로 신체(외수, 의족, 의안. 의치 등 신체보조장구는 제외하나, 인공장기나 부분 의치 등 신체에 이식되어 그 기능을 대신할 경우는 포함합니다)에 입은 상해를 말합니다. 나. 장해: 「장해분류표」 ( 별별1) 참조) 및 「장해분류표II」 ( 별별2 ) 참조)에서 정한 기준에 따른 장해상태를 말합니다. 다. 중요한 사항: 계약전 알릴 의무와 관련하여 회사가 그 사실을 알았더라면 계약의 청약을 거절하거나 보험가입금액 한도 제한, 일부 보장 제외, 보험금 삭감, 보험료 할증과 같이 조건부로 승낙하는 등 계약 승낙에 영향을 미칠 수 있는 사항을 말합니다. 3. 이자율 관련 용어 가. 연단위 복리: 회사

In [10]:
len(string_result)

68985

In [11]:
## Chunking
def sliding_window_chunking(text, window_size, step_size):
    chunks = []
    for i in range(0, len(text) - window_size + 1, step_size):
        chunk = text[i:(i + window_size)]
        chunks.append(chunk)

    if len(text) % window_size != 0:
        chunks.append(text[-window_size])

    return chunks

In [12]:
window_size = 50
step_size = 10

text = "이 텍스트는 슬라이딩 윈도우 청킹을 사용해 나눠진다. 이 청킹 방식은 텍스트의 연속성을 유지하면서 적절한 크기의 청크로 나누는 방법이다."
chunks = sliding_window_chunking(text, window_size, step_size)
for i, chunk in enumerate(chunks, 1):
    print(f"청크 {i} : {chunk}")

청크 1 : 이 텍스트는 슬라이딩 윈도우 청킹을 사용해 나눠진다. 이 청킹 방식은 텍스트의 연속성을 유
청크 2 : 딩 윈도우 청킹을 사용해 나눠진다. 이 청킹 방식은 텍스트의 연속성을 유지하면서 적절한 크
청크 3 : 사용해 나눠진다. 이 청킹 방식은 텍스트의 연속성을 유지하면서 적절한 크기의 청크로 나누는
청크 4 : 진


In [13]:
df = pd.read_excel('./data/corpus.xlsx')

In [14]:
string_result = df['text'][0]

In [ ]:
## 689585 --> 32767로 줄어들었음. 이는 excel로 저장했기 때문.
## 따라서 가급적이면 .txt나 .json으로 저장하는 것을 권장.

len(string_result)

32767

In [ ]:
## 문장 단위 스플릿(sentence-wise chunking)
## 다만 OCR로 글자를 추출하다보니 한 문장이 올바르게 구성되어 있지 않는 경우가 있음.
## 따라서 현재 데이터를 대상으로 문장 단위 청킹은 적절한 청킹 방법이라 보기 어려움.
from kss import Kss

split_sentences = Kss("split_sentences")
split_sentences(string_result)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



['보험약관 카드회원단체상해보험 목차 보통약관 특별약관 해외여행중 배상책임 특별약관 국내여행중 공공교통승용구탑승중 배상책임 특별약관 상해사망후유장해 부보장 특별약관 지정대리청구서비스 특별약관 카드회원 보험료정산 추가약관 해외발생 실손의료비(카드회단체) 특별약관 【보통약관】 제1관 목적 및 용어의 정의 제1조(목적) 이 보험계약(이하 \'계약\' 이라 합니다)은 보험계약자(이하 \'계약자\' 라 합니다)와 보험회사(이하 \'회사\' 라 합니다) 사이에 피보험자의 상해에 대한 위험을 보장하기 위하여 체결됩니다. 제2조(용어의 정의) 이 계약에서 사용되는 용어의 정의는, 이 계약의 다른 조항에서 달리 정의되지 않는 한 다음과 같습니다. 1. 계약관계 관련 용어 가. 계약자: 회사와 계약을 체결하고 보험료를 납입할 의무를 지는 사람을 말합니다. 나. 보험수익자: 보험금 지급사유가 발생하는 때에 회사에 보험금을 청구하여 받을 수 있는 사람을 말합니다. 다. 보험증권: 계약의 성립과 그 내용을 증명하기 위하여 회사가 계약자에게 드리는 증서를 말합니다. 라. 진단계약: 계약을 체결하기 위하여 피보험자가 건강진단을 받아야 하는 계약을 말합니다. 마. 피보험자: 보험사고의 대상이 되는 사람을 말합니다. 2. 지급사유 관련 용어 가. 상해: 보험기간 중에 발생한 급격하고도 우연한 외래의 사고로 신체(외수, 의족, 의안. 의치 등 신체보조장구는 제외하나, 인공장기나 부분 의치 등 신체에 이식되어 그 기능을 대신할 경우는 포함합니다)에 입은 상해를 말합니다. 나. 장해: 「장해분류표」 ( 별별1) 참조) 및 「장해분류표II」 ( 별별2 ) 참조)에서 정한 기준에 따른 장해상태를 말합니다. 다. 중요한 사항: 계약전 알릴 의무와 관련하여 회사가 그 사실을 알았더라면 계약의 청약을 거절하거나 보험가입금액 한도 제한, 일부 보장 제외, 보험금 삭감, 보험료 할증과 같이 조건부로 승낙하는 등 계약 승낙에 영향을 미칠 수 있는 사항을 말합니다. 3. 이자율 관련 용어 가. 연단위 복리: 회

In [18]:
split_result = split_sentences(string_result)

In [19]:
len(split_result)

101

In [20]:
df_splited = pd.DataFrame({'chunk' : split_result,
                           'chunk_no' : list(range(len(split_result)))})

In [ ]:
df_splited.to_excel('./data/chunk_corpus.xlsx', index=False)

In [22]:
df_splited['len_chunk']=df_splited['chunk'].str.len()

In [23]:
df_splited

,chunk,chunk_no,len_chunk
0,보험약관 카드회원단체상해보험 목차 보통약관 특별약관 해외여행중 배상책임 특별약관 국...,0,14401
1,"생년월일: 1988년 10월 2일, 현재(계약일: 2014년 4월 13일 2014년...",1,8575
2,제3조(보험금의 지급사유) 1 회사는 피보험자가 보통약관 제3조(보험금의 지급사유)...,2,120
3,"또는 재물의 손해(재물의 없어짐, 손상 및 망가짐을 포함합니다)에 대한 법률상의 배...",3,84
4,2 회사가 1사고당 보상하는 손해의 범위는 아래와 같습니다.,4,33
5,1. 피보험자가 피해자에게 지급할 책임을 지는 법률상의 손해배상금 2. 계약자 또는...,5,66
6,피보험자가 제11조(손해방지의무) 제1항 제1호의 손해의 방지 또는 경감을 위하여 ...,6,67
7,피보험자가 제11조(손해방지의1무) 제1항 제2호의 제3자로부터 손해의 배상을 받을...,7,140
8,보험증권상의 보상한도액내의 금액에 대한 공탁보증보험료.,8,30
9,그러나 회사는 그러한 보증을 제공할 책임은 부담하지 않습니다.,9,34


In [24]:
## 라마인덱스의 TokenTextSplitter로 분할.

from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [25]:
chunker = TokenTextSplitter(chunk_size=512, chunk_overlap=0)
document = [Document(text=string_result)]

result = chunker.get_nodes_from_documents(documents=document)
chunked_texts = list(map(lambda x: x.text, result))

In [26]:
chunked_texts

["보험약관 카드회원단체상해보험 목차 보통약관 특별약관 해외여행중 배상책임 특별약관 국내여행중 공공교통승용구탑승중 배상책임 특별약관 상해사망후유장해 부보장 특별약관 지정대리청구서비스 특별약관 카드회원 보험료정산 추가약관 해외발생 실손의료비(카드회단체) 특별약관 【보통약관】 제1관 목적 및 용어의 정의 제1조(목적) 이 보험계약(이하 '계약' 이라 합니다)은 보험계약자(이하 '계약자' 라 합니다)와 보험회사(이하 '회사' 라 합니다) 사이에 피보험자의 상해에 대한 위험을 보장하기 위하여 체결됩니다. 제2조(용어의 정의) 이 계약에서 사용되는 용어의 정의는, 이 계약의 다른 조항에서 달리 정의되지 않는 한 다음과 같습니다. 1. 계약관계 관련 용어 가. 계약자: 회사와 계약을 체결하고 보험료를 납입할 의무를 지는 사람을 말합니다. 나. 보험수익자: 보험금 지급사유가 발생하는 때에 회사에 보험금을 청구하여 받을 수 있는 사람을 말합니다. 다. 보험증권:",
 '계약의 성립과 그 내용을 증명하기 위하여 회사가 계약자에게 드리는 증서를 말합니다. 라. 진단계약: 계약을 체결하기 위하여 피보험자가 건강진단을 받아야 하는 계약을 말합니다. 마. 피보험자: 보험사고의 대상이 되는 사람을 말합니다. 2. 지급사유 관련 용어 가. 상해: 보험기간 중에 발생한 급격하고도 우연한 외래의 사고로 신체(외수, 의족, 의안. 의치 등 신체보조장구는 제외하나, 인공장기나 부분 의치 등 신체에 이식되어 그 기능을 대신할 경우는 포함합니다)에 입은 상해를 말합니다. 나. 장해: 「장해분류표」 ( 별별1) 참조) 및 「장해분류표II」 ( 별별2 ) 참조)에서 정한 기준에 따른 장해상태를 말합니다. 다. 중요한 사항: 계약전 알릴 의무와 관련하여 회사가 그 사실을 알았더라면 계약의 청약을 거절하거나 보험가입금액 한도 제한, 일부 보장 제외, 보험금 삭감, 보험료 할증과 같이 조건부로 승낙하는 등 계약 승낙에 영향을 미칠 수 있는 사항을 말합니다. 3. 이자율 관련 용어 가. 연단위 복리: 회사가